## Recurrent neural network 
- >Image classification
    -  split traindata and testdata
    -  handwrite number classification
    -  put train/test data to Dataloader(shuffle,batch_size,num_workers) respectively


In [23]:
import torch
import torchvision
import torch.nn as nn                        # Activation Funciton , Algo
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.utils.data as Data              # Data loader 

In [11]:
# Hyper parameters
EPOCH = 1
BATCH_SIZE = 64
TIME_STEP = 28      # RNN 考慮多少時間點的數據     [image height]
INPUT_SIZE = 28     # RNN 每一個時間需要多少數據點 [image width]
LR = 0.01
DOWNLOAD_MNIST = False


## Datasets
- DataLoader
- Training and Testing datasets

In [12]:
train_dataset = dsets.CIFAR10(
    root = './CIFAR10',
    train = True,
    transform = transforms.ToTensor(),
    download = DOWNLOAD_MNIST,
)

train_dataloader = Data.DataLoader(
    batch_size = BATCH_SIZE,
    num_workers = 2,
    shuffle = True,
    dataset = train_dataset
)


test_dataset = dsets.CIFAR10(
    root = './CIFAR10',
    train = False,
    transform = transforms.ToTensor(),
    download = DOWNLOAD_MNIST
)

test_dataloader = Data.DataLoader(
    batch_size = BATCH_SIZE,
    num_workers = 2,
    shuffle = True ,
    dataset = test_dataset

)

In [26]:
x ,y  = next(iter(train_dataloader))
x

tensor([[[[0.4000, 0.3882, 0.3843,  ..., 0.3255, 0.3255, 0.3255],
          [0.3765, 0.3647, 0.3608,  ..., 0.3098, 0.3098, 0.3137],
          [0.3647, 0.3529, 0.3490,  ..., 0.2902, 0.2941, 0.3020],
          ...,
          [0.1804, 0.1804, 0.1882,  ..., 0.2000, 0.2000, 0.2039],
          [0.1765, 0.1765, 0.1765,  ..., 0.2000, 0.2000, 0.2039],
          [0.1804, 0.1804, 0.1843,  ..., 0.2039, 0.2000, 0.2039]],

         [[0.5255, 0.5098, 0.5059,  ..., 0.4784, 0.4784, 0.4824],
          [0.4980, 0.4824, 0.4784,  ..., 0.4667, 0.4667, 0.4745],
          [0.4824, 0.4706, 0.4667,  ..., 0.4510, 0.4549, 0.4627],
          ...,
          [0.3725, 0.3686, 0.3725,  ..., 0.3843, 0.3843, 0.3922],
          [0.3686, 0.3647, 0.3647,  ..., 0.3843, 0.3843, 0.3882],
          [0.3765, 0.3725, 0.3725,  ..., 0.3882, 0.3882, 0.3922]],

         [[0.7490, 0.7333, 0.7373,  ..., 0.7373, 0.7333, 0.7451],
          [0.7333, 0.7176, 0.7216,  ..., 0.7176, 0.7176, 0.7294],
          [0.7294, 0.7176, 0.7216,  ..., 0

In [33]:
t[:,-1,:]

tensor([[0.6588, 0.0759, 0.6837],
        [0.6110, 0.5740, 0.3035],
        [0.0759, 0.1919, 0.8848]])

## Main program
- construct RNN

In [41]:
class RNN(nn.Module):
    def __init__(self) -> None:
        super(RNN , self).__init__()

        self.rnn = nn.LSTM(
            input_size = INPUT_SIZE,    # 每一個時間點有多少數據
            hidden_size = 64 , 
            num_layers = 1 ,           # 多少層的 hidden layer
            batch_first = True          # 傳進來的數據維度是否是 batch_size 在最前面 [batch_size , time_step , input_size] , batch_size 不是在前面的話就是 False
        )

        self.out = nn.Linear(64 , 10)

    def forward(self , x):

        # 會分成 rnn_out 跟 hidden_state
        # 時間 t+1 狀態會根據 時間 t 所產生的 hidden state 去產生 t+1 的 rnn_out 
        # ( h_n , h_c )為 都為 hidden state ， 為 LSTM 的 主線及分線記憶 
        rnn_out , (h_n , h_c) = self.rnn(x , None) # x (batch_size[64] , time_step[28] , input_size[28])  , None -> 傳入 rnn 之前有沒有 hidden_state
        output = self.out(rnn_out[:,-1,:])         # 取得 time_step 最後 ， 因為前一部都是根據上一部的 time_step 得到 rnn_out 
        
        return output
        


rnn = RNN()
rnn


RNN(
  (rnn): LSTM(28, 64, batch_first=True)
  (out): Linear(in_features=64, out_features=10, bias=True)
)